# Détermination du parcours client : Notebook Exhaustif

Ce notebook a pour but de faire partager le cheminement que nous avons mené tout au long de notre projet pour arriver au pipeline final.

Vous y trouverez donc pas mal de statistiques exploratoires mais aussi des essais de modélisation non fructueux ou bien des tests de paramètres pour les modèles finaux.

## Importation des données

Commencons par importer les données :
- make_data :
    - crée les dossiers qui contiendront les données,
    - charge les données "raw" (pour le moment fait à la main).
    
- clean_data :
    - met des labels aux colonnes, 
    - supprime les données redondates,
    - reformate les timestamps,
    - sauvegarde ça dans un csv et un dataframe : "buyer_history".

In [1]:
%run ../src/make_data.py
%run ../src/clean_data.py

Mettez les export_ensai_buyersHistory_170216.csv et export_ensai_buyersProfil_170216.csv et PFEensai2017_description_des_donnees.xlsx dans PFE/data/raw/


## Statistique Exploratoire

### Comportements des acheteurs (sans visites)

Regardons tout d'abord comment les acheteurs se comportent indépendamment des visites.

Déjà on peut regarder si les visites sont significatives ou pas :

In [2]:
buyer_history.drop_duplicates().shape[0] - buyer_history[["buyer_id","event","timestamp","status"]].drop_duplicates().shape[0]

4063

On peut voir que certaines lignes sont redondantes si on enlève les visites. Cela veut dire qu'un personne effectue deux fois la même action au même moment mais avec des visit_id différentes ?

Ce genre de lignes polluent notre base de données et doivent être enlevées.

Maintenant regardons le nombre d'évenements pas achteurs.

In [3]:
buyer_history.groupby("buyer_id")["event"].count().describe()

count    14654.000000
mean         6.932442
std        146.689088
min          1.000000
25%          2.000000
50%          2.000000
75%          4.000000
max      17661.000000
Name: event, dtype: float64

On peut voir que le nombre d'event median par buyer est de 2 mais que la moyenne est de 6.9 : distribution asymétrique.

Le nombre d'event max pour un acheteur est de 17837 : appelons cette personne mec_chelou et regardons son comportement:

In [4]:
mec_chelou = buyer_history.groupby("buyer_id")["event"].count().idxmax()
buyer_history[buyer_history["buyer_id"]==mec_chelou]["event"].value_counts()[0:9]

pageview                     10279
iadvize_displayed             2931
demande_de_catalogue           981
telechargement_3d              873
visit_identified               604
formulaire_web                 548
crm_email                      462
demande_de_devis               170
crm_Courrier_électronique      135
Name: event, dtype: int64

On peut voir plus de 10000 page views

In [5]:
((buyer_history["buyer_id"]==mec_chelou) & (buyer_history["event"]=="projet") & (buyer_history["status"]=="Gagné")).sum()

19

Seulement 19 projet acheté : C'est peut être le compte d'une entreprise ou simplement d'un employé du magasin réalisant des projets pour ses clients.

Pour le moment il n'est pas représentatif de la population et peut grandement fausser des futurs résultats : on préfère l'enlever pour le moment.

Ce changement et la suppresion des données redondantes se font dans explore_data.py

In [6]:
%run ../src/explore_data.py

Continuons en regardant les 20 autres suivants pour voir s'ils ont eux aussi un comportement anormal.

On va regarder leur pageview et leur achat principalement et leur annulation voir s'ils ont hésitants :

In [7]:
top20_chelou = buyer_history.groupby("buyer_id")["event"].count().sort_values(ascending=False)[0:19]
for chelou in top20_chelou.index:
    print("pageview = {0}".format(((buyer_history["buyer_id"]==chelou) & (buyer_history["event"]=="pageview")).sum()),end=", ")
    print("achat = {0}".format(((buyer_history["buyer_id"]==chelou) & (buyer_history["event"]=="projet_win")).sum()),end=", ")
    print("gestion du projet = {0}".format(((buyer_history["buyer_id"]==chelou) & (buyer_history["event"]=="projet")).sum()))

pageview = 420, achat = 0, gestion du projet = 0
pageview = 283, achat = 0, gestion du projet = 0
pageview = 280, achat = 0, gestion du projet = 0
pageview = 216, achat = 0, gestion du projet = 0
pageview = 166, achat = 0, gestion du projet = 0
pageview = 163, achat = 0, gestion du projet = 0
pageview = 140, achat = 0, gestion du projet = 0
pageview = 152, achat = 0, gestion du projet = 0
pageview = 143, achat = 0, gestion du projet = 0
pageview = 160, achat = 0, gestion du projet = 0
pageview = 128, achat = 0, gestion du projet = 0
pageview = 144, achat = 0, gestion du projet = 0
pageview = 179, achat = 0, gestion du projet = 0
pageview = 193, achat = 0, gestion du projet = 0
pageview = 167, achat = 0, gestion du projet = 0
pageview = 158, achat = 0, gestion du projet = 0
pageview = 152, achat = 0, gestion du projet = 0
pageview = 134, achat = 0, gestion du projet = 0
pageview = 119, achat = 0, gestion du projet = 0


On a l'impression qu'on est face a des acheteurs hésitants plutôt que des données fausses : on va les garder.

### Acheteurs dans le temps

Petite info sur le nombre de visite par buyer_id

In [8]:
buyer_history[["buyer_id","visit_id"]].drop_duplicates().groupby(["buyer_id"]).count().describe()

,visit_id
count,14653.000000
mean,3.412544
std,3.431968
min,1.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,72.000000


On va regarder maintenant les timestamps maximums et minimums :

In [9]:
max_ts = buyer_history[["buyer_id","timestamp"]].groupby(["buyer_id"]).max()
min_ts = buyer_history[["buyer_id","timestamp"]].groupby(["buyer_id"]).min()

min_ts["date"]=stringlist_to_datelist(min_ts["timestamp"])
max_ts["date"]=stringlist_to_datelist(max_ts["timestamp"])

Maintenant on peut regarder la distribution du temps de passage par acheteur.

In [10]:
(max_ts["date"]-min_ts["date"]).describe()

count                      14653
mean     23 days 09:05:11.661161
std      58 days 10:35:30.251437
min              0 days 00:00:00
25%              0 days 00:00:00
50%              0 days 00:00:00
75%              6 days 01:49:00
max            495 days 10:43:44
Name: date, dtype: object

### Les évènements

Un petit regard sur les évènements en eux memes.

In [11]:
buyer_history["event"].value_counts()

pageview          24983
store             16090
project_win       14735
chat               6525
appointement       5740
download           2045
email              1945
catalogue          1856
idenfitication     1354
form               1209
sms                1087
project             834
quote               368
mail                360
call                329
project_lose        268
sms_lose            120
task                 85
contact              37
newsletter           20
catalogue_lose       17
quote_lose            3
other                 2
Name: event, dtype: int64

Et voilà !